Testing enrichment for genesets in the rare variant paper Nature. 2014 Feb 13; 506(7487): 185–190.

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from bsmcalls import SNPnexus
from bsmcalls import operations
from matplotlib import pyplot as plt
import attila_utils
%matplotlib inline

In [3]:
data = SNPnexus.load_data('/home/attila/projects/bsm/results/2020-09-07-annotations/annotated-calls.p')

In [48]:
fpath = '/home/attila/projects/bsm/resources/rare-variants/41586_2014_BFnature12975_MOESM71_ESM.csv'
genesets = pd.read_csv(fpath, sep='\t', index_col='Gene symbol').dropna().astype(pd.CategoricalDtype())
genesets = genesets.apply(lambda s: s.cat.rename_categories({'.': False, 'Y': True}), axis=1)
genesets['composite'] = True
genesets

,PSD,FMRP-target,SCZ de novo,Calcium channel,composite
Gene symbol,,,,,
AAK1,False,True,False,False,True
AARS,True,False,False,False,True
AATK,False,True,False,False,True
ABCA2,False,True,False,False,True
ABCA3,False,True,False,False,True
...,...,...,...,...,...
ZNF721,False,False,True,False,True
ZNF77,False,False,True,False,True
ZNFX1,False,True,False,False,True


In [49]:
genesetsd = {c: set(genesets.index[genesets[c]]) for c in genesets}
querydict = {'near_gens_Annotation': ['coding nonsyn', 'coding syn', 'missense', 'stop-gain', 'intronic (splice_site)'],
             'near_gens_Overlapped Gene': genesetsd,
            }
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)

In [50]:
results_vtype = results.xs('near_gens_Annotation', axis=1)[['coding nonsyn', 'coding syn']]
results_constr = results.xs('near_gens_Overlapped Gene', axis=1)

l = [pd.DataFrame({constr: results_vtype[vtype] & results_constr[constr] for constr in results_constr}) for vtype in results_vtype]

results_combi = pd.concat(l, axis=1)
results_combi.columns = pd.MultiIndex.from_product([results_vtype.columns, results_constr.columns])

In [51]:
summary = operations.summarize_query_results(pd.concat([results, results_combi], axis=1), data)
summary

/home/attila/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/home/attila/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:5982: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


Control  SCZ  ASD  All  \
Feature                   Query                                            
near_gens_Annotation      coding nonsyn                23   43   23   89   
                          coding syn                   16   20   15   51   
                          missense                      0    0    0    0   
                          stop-gain                     1    0    0    1   
                          intronic (splice_site)        0    1    0    1   
near_gens_Overlapped Gene PSD                          42   70   71  183   
                          FMRP-target                  88  139  114  341   
                          SCZ de novo                  43   69   69  181   
                          Calcium channel               2    9    2   13   
                          composite                   140  237  192  569   
coding nonsyn             PSD                           2    0    0    2   
                          FMRP-target                   2    2    3    7   
                          SCZ de novo                   2    2    4    8   
                          Calcium channel               0    1    0    1   
                          composite                     5    5    7   17   
coding syn                PSD                           1    1    2    4   
                          FMRP-target                   1    3    0    4   
                          SCZ de novo                   1    1    1    3   
                          Calcium channel               0    0    0    0   
                          composite                     1    4    3    8   

                                                  chisq stat   chisq p  
Feature                   Query                                         
near_gens_Annotation      coding nonsyn             4.480340  0.106440  
                          coding syn                1.177849  0.554924  
                          missense                       NaN       NaN  
                          stop-gain                 2.887477  0.236044  
                          intronic (splice_site)    1.582797  0.453210  
near_gens_Overlapped Gene PSD                       1.097207  0.577756  
                          FMRP-target               0.803238  0.669236  
                          SCZ de novo               0.623119  0.732304  
                          Calcium channel           5.154270  0.075991  
                          composite                 2.069647  0.355289  
coding nonsyn             PSD                       5.774955  0.055717  
                          FMRP-target               0.313055  0.855108  
                          SCZ de novo               0.859645  0.650625  
                          Calcium channel           1.582797  0.453210  
                          composite                 0.621018  0.733074  
coding syn                PSD                       0.429823  0.806613  
                          FMRP-target               2.783164  0.248682  
                          SCZ de novo               0.094176  0.954003  
                          Calcium channel                NaN       NaN  
                          composite                 0.815315  0.665207

In [2]:
%connect_info

{
  "shell_port": 55089,
  "iopub_port": 44629,
  "stdin_port": 57407,
  "control_port": 54101,
  "hb_port": 45817,
  "ip": "127.0.0.1",
  "key": "1b847f2d-6c79c0c9c9f33bc6e1171f04",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-63c244bb-5234-4905-a909-ad56793c2fbc.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
